# Set up

In [ ]:
! pip install -q -U datasets
! pip install -q transformers -U
! pip install -q transformers[torch] -U
! pip install -q sentencepiece
! pip install -q evaluate
! pip install -q bert_score
! pip install -q -U accelerate
! pip install -q peft
! pip install -q -U huggingface-hub
! pip install -q sentence-transformers

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

# CovidQA

In [ ]:
from datasets import load_dataset

dataset_name = "minh21/COVID-QA-Chunk-64-question-answering-biencoder-data-90_10"
dataset = load_dataset(dataset_name, use_auth_token=True)

test_set_name = "minh21/COVID-QA-Chunk-64-testset-biencoder-data-90_10"
dataset["test"] = load_dataset(test_set_name, use_auth_token=True, split="train")

dataset = dataset.remove_columns(["id"])

In [ ]:
from sentence_transformers import CrossEncoder

top_k = 10
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

In [6]:
def retrieve_context(row):
    query = row["question"]

    # cross-encoder
    hits = [{"corpus_id": i} for i in range(len(row["context_chunks"]))]
    cross_inp = [[query, row["context_chunks"][hit["corpus_id"]]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp, show_progress_bar=False)

    for idx in range(len(cross_scores)):
        hits[idx]["cross-score"] = cross_scores[idx]

    hits = sorted(hits, key=lambda x: x["cross-score"], reverse=True)

    row["retrieved_context"] = " ".join(
        [row["context_chunks"][hit["corpus_id"]] for hit in hits[0:top_k]]
    ).replace("\n", " ")

    return row

In [ ]:
dataset = dataset.map(retrieve_context, batched=False)

In [8]:
match = 0


def find_match(row):
    global match
    match = match + (1 if row["answer"] in row["retrieved_context"] else 0)

In [ ]:
match = 0
dataset["train"].map(find_match, batched=False)
print("Train: %.2f" % (match / dataset["train"].num_rows * 100))

match = 0
dataset["validation"].map(find_match, batched=False)
print("Validation: %.2f" % (match / dataset["validation"].num_rows * 100))

match = 0
dataset["test"].map(find_match, batched=False)
print("Test: %.2f" % (match / dataset["test"].num_rows * 100))

In [10]:
dataset["test"] = dataset["test"].remove_columns(["context"])
dataset = dataset.rename_columns({"retrieved_context": "context"})

In [ ]:
dataset["train"] = dataset["train"].filter(lambda row: row["answer"] in row["context"])
dataset["validation"] = dataset["validation"].filter(
    lambda row: row["answer"] in row["context"]
)
dataset

In [ ]:
dataset.save_to_disk("./covid-qa-ce-100%")

In [3]:
from datasets import DatasetDict

dataset = DatasetDict.load_from_disk("./covid-qa-ce-100%")

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'context_chunks', 'document_id', 'context'],
        num_rows: 1480
    })
    validation: Dataset({
        features: ['question', 'answer', 'context_chunks', 'document_id', 'context'],
        num_rows: 171
    })
    test: Dataset({
        features: ['question', 'answer', 'context_chunks', 'document_id', 'context'],
        num_rows: 203
    })
})

# Model

In [16]:
model_nm = "google/flan-t5-large"
max_length = 512
max_target_length = 200
checkpoint = "flan-t5-large-ia3-covidqa"
username = "legacy107"

In [8]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
import evaluate


def evaluate_qa(predicted_result):

    squad_metric = evaluate.load("squad")
    predictions = [
        {"prediction_text": answer, "id": str(id)}
        for id, answer in enumerate(predicted_result["output"])
    ]
    references = [
        {
            "answers": {"answer_start": [-1], "text": [ds["answer"]]},
            "id": str(id),
        }
        for id, ds in enumerate(predicted_result)
    ]
    results = {}
    results["squad"] = squad_metric.compute(
        predictions=predictions, references=references
    )

    predictions = predicted_result["output"]
    references = predicted_result["answer"]

    bleu_metrics = evaluate.load("bleu")
    results["bleu"] = bleu_metrics.compute(
        predictions=predictions, references=references
    )

    bertscore_metric = evaluate.load("bertscore")
    berscore = bertscore_metric.compute(
        predictions=predictions, references=references, lang="en"
    )
    results["bertscore"] = {
        "precision": sum(berscore["precision"]) / len(berscore["precision"]),
        "recall": sum(berscore["recall"]) / len(berscore["recall"]),
        "f1": sum(berscore["f1"]) / len(berscore["f1"]),
    }

    return results

# Evaluate prompting

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

tokenizer = AutoTokenizer.from_pretrained(model_nm, device_map="auto")
model = AutoModelForSeq2SeqLM.from_pretrained(model_nm, device_map="auto")

In [ ]:
def interfere(ds):
    ds["input"] = f"question: {ds['question']} " f"context: {ds['context']}"
    input_ids = tokenizer(
        ds["input"],
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=max_length,
    ).input_ids.to(torch.device("cuda"))
    outputs = model.generate(input_ids, max_new_tokens=max_target_length)
    ds["output"] = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return ds


model.eval()
with torch.no_grad():
    predicted_result = dataset["test"].map(interfere, batched=False)

In [ ]:
evaluate_qa(predicted_result)

# IA3

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModelForSeq2SeqLM, get_peft_config

config = {
    "peft_type": "IA3",
    "task_type": "SEQ_2_SEQ_LM",
    "inference_mode": False,
    "target_modules": ["q", "v", "wo"],
    "feedforward_modules": ["wo"],
}

peft_config = get_peft_config(config)
tokenizer = AutoTokenizer.from_pretrained(model_nm)
model = AutoModelForSeq2SeqLM.from_pretrained(model_nm)
peft_model = PeftModelForSeq2SeqLM(model, peft_config)
peft_model.print_trainable_parameters()

In [ ]:
def qa_prompt_preprocess(ds):
    input = f"question: {ds['question']} " f"context: {ds['context']}"
    model_inputs = tokenizer(input, max_length=max_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(
        text_target=ds["answer"], max_length=max_target_length, truncation=True
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


seq2seq_dataset = {
    "train": dataset["train"].map(qa_prompt_preprocess, batched=False),
    "test": dataset["validation"].map(qa_prompt_preprocess, batched=False),
}

In [14]:
seq2seq_dataset["train"] = seq2seq_dataset["train"].remove_columns(
    ["answer", "document_id", "context_chunks", "question", "context"]
)
seq2seq_dataset["test"] = seq2seq_dataset["test"].remove_columns(
    ["answer", "document_id", "context_chunks", "question", "context"]
)

In [ ]:
squad_metric = evaluate.load("squad")
bleu_metrics = evaluate.load("bleu")


def compute_metrics(pred):
    labels_ids, pred_ids = pred

    pred_ids[pred_ids == -100] = tokenizer.pad_token_id
    predictions = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)

    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    references = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    squad_predictions = [
        {"prediction_text": answer, "id": str(id)}
        for id, answer in enumerate(predictions)
    ]
    squad_references = [
        {
            "answers": {"answer_start": [-1], "text": [answer]},
            "id": str(id),
        }
        for id, answer in enumerate(references)
    ]
    results = {}
    results["squad"] = squad_metric.compute(
        predictions=squad_predictions, references=squad_references
    )

    results["bleu"] = bleu_metrics.compute(
        predictions=predictions, references=references
    )

    return results

In [16]:
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
from transformers.trainer_callback import TrainerCallback
import os


class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args,
        state,
        control,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(
            args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}"
        )

        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path)

        pytorch_model_path = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_model_path):
            os.remove(pytorch_model_path)
        return control

In [17]:
from transformers import (
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)

bs = 2
eval_bs = 1
epochs = 16
lr = 3e-3
steps = 200
max_steps = 4000  # number of steps before overfitting
training_args = Seq2SeqTrainingArguments(
    output_dir=f"./{checkpoint}",
    learning_rate=lr,
    # per_device_train_batch_size=bs,
    # per_device_eval_batch_size=eval_bs,
    auto_find_batch_size=True,
    #     num_train_epochs=epochs,
    max_steps=4000,
    weight_decay=0.01,
    lr_scheduler_type="linear",
    push_to_hub=True,
    save_strategy="steps",
    save_steps=steps,
    logging_steps=steps,
    evaluation_strategy="steps",
    eval_steps=steps,
    predict_with_generate=True,
    generation_max_length=max_target_length,
)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=peft_model,
    padding="max_length",
    max_length=max_length,
    return_tensors="pt",
)

trainer = Seq2SeqTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=seq2seq_dataset["train"].with_format("torch"),
    eval_dataset=seq2seq_dataset["test"].with_format("torch"),
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[SavePeftModelCallback],
)

In [ ]:
trainer.train()

In [19]:
trainer.model.save_pretrained(checkpoint)
tokenizer.save_pretrained(checkpoint)

trainer.model.push_to_hub(checkpoint)
tokenizer.push_to_hub(checkpoint)

('flan-t5-large-ia3-covidqa2/tokenizer_config.json',
 'flan-t5-large-ia3-covidqa2/special_tokens_map.json',
 'flan-t5-large-ia3-covidqa2/spiece.model',
 'flan-t5-large-ia3-covidqa2/added_tokens.json',
 'flan-t5-large-ia3-covidqa2/tokenizer.json')

## Evaluate from checkpoint

In [ ]:
from transformers import AutoTokenizer
from peft import AutoPeftModelForSeq2SeqLM

model_checkpoint = f"{username}/{checkpoint}"
model_checkpoint = "/kaggle/working/flan-t5-large-ia3-covidqa2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, device_map="auto")
checkpoint_model = AutoPeftModelForSeq2SeqLM.from_pretrained(
    model_checkpoint, device_map="auto"
)

In [ ]:
import torch


def interfere_prompt(ds):
    qa_input = f"question: {ds['question']} " f"context: {ds['context']}"
    input_ids = tokenizer(
        qa_input,
        return_tensors="pt",
        truncation=True,
        max_length=max_length,
    ).to(torch.device("cuda"))
    outputs = checkpoint_model.generate(**input_ids, max_new_tokens=max_target_length)
    ds["output"] = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return ds


checkpoint_model.eval()
with torch.no_grad():
    predicted_result = dataset["test"].map(interfere_prompt, batched=False)

In [ ]:
evaluate_qa(predicted_result)